### Финальный проект

Мы уже прошли всю необходимуб теорию для финального проекта. Проект осуществляется на данных из вебинара (данные считаны в начале ДЗ).
Рекомендуем вам **начать делать проект сразу после этого домашнего задания**  
- Целевая метрика - precision@5. Порог для уcпешной сдачи проекта precision@5 > 25%  
- Будет public тестовый датасет, на котором вы сможете измерять метрику  
- Также будет private тестовый датасет для измерения финального качества  
- НЕ обязательно, но крайне желательно использовать 2-ух уровневые рекоммендательные системы в проекте  
- Вы сдаете код проекта в виде github репозитория и csv файл с рекомендациями    

In [ ]:
отбор кандидатов. 
15- из собственных товаровб
30 - и списка наиболее популярных 
то 50  товаров из mainrecomender getalsrecomendadions.


фитчи на основе со встречаемости товаров. 
с этим товаро покупают.
--> lightGBM ранжирует товары. и выбирает из них топ 30

--> к полученному набору применим постфильтрацию.